In [2]:
# using Pkg
# Pkg.activate("/home/jek354/research/ML-signproblem")
# Pkg.update()

using Lattices
using LinearAlgebra
using Combinatorics
using SparseArrays
using Plots
import Graphs
using LaTeXStrings
using Statistics
using Random
using Zygote
using Optimization, OptimizationOptimisers
using JSON
using OptimizationOptimJL
using JLD2
using KrylovKit
# using ExponentialUtilities


include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")
# include("adiabatic_analysis.jl")

merge_jld2_folder

In [54]:
# using KrylovKit


t = 1.0
U = 6
μ = 0  # positive incentivises fewer particles (one electron costs this much energy)
N_up = 4
N_down = 4
N =  4
half_filling = false
# lattice = Chain(6, Periodic())
lattice_dimension = (4,2)
bc = "periodic"
# lattice = Chain(6, Periodic())
lattice = Square(lattice_dimension, if bc == "periodic" Periodic() else Open() end)
# lattice = Graphs.cycle_graph(3)

models = []

reference_index = 2
# t_values = []# Float64[1,1], Float64[1,0.1], 1.0]
# for _t in t_values
#     # println(_t)
#     push!(models,HubbardModel(_t,0.0001,μ,half_filling))
# end
# U_values = [0.00001,0.01]
U_values = [0.00001; LinRange(2.1,9,20)]
U_values = sort([U_values; 10.0 .^LinRange(-3,2,40)])

for U in U_values
    # println(t)
    push!(models,HubbardModel(t,U,μ,half_filling))
end

subspace = HubbardSubspace(N, lattice)
# subspace = HubbardSubspace(N_up, N_down, lattice)

# ops = []
# if subspace.N >= 1
#     push!(ops,Matrix(create_operator(subspace,:Sx)))
# end
# push!(ops, Matrix(create_operator(subspace,:S2)))
# # op3 = Matrix(create_operator(subspace,:L2))
# push!(ops, Matrix(create_operator(subspace,:T, kind=1)))
# push!(ops, Matrix(create_operator(subspace,:T, kind=2)))
# push!(ops, Matrix(create_operator(subspace,:σ, kind=1)))
E = []
H = []
V = []
# for model ∈ models
#     push!(H, Matrix(create_Hubbard(model, subspace; perturbations=false)))
#     e, v = eigen(H[end])
#     push!(E, e)
#     push!(V, v)
# end

H, indexer = create_Hubbard(models[20], subspace; get_indexer=true)

# eigsolve(H, normalize(rand(size(H)[1])), 1,:SR)

# for i ∈ 1:2
#     println(degeneracy_count(E[i]))
# end

# println(sum(abs.(h1*h2-h2*h1)))

# ops_tmp = [H[1], ops...]
# for i in eachindex(ops_tmp)
#     for j in i+1:length(ops_tmp)
#         println("($i,$j) ", sum(abs.(ops_tmp[i]*ops_tmp[j] - ops_tmp[j]*ops_tmp[i])))
#     end
# end

# H1 = create_Hubbard(model, subspace; perturbations=false)
# E1 = diag(V1'*Matrix(H1)*V1)

# @time degen_rm_U = create_consistent_basis(H, ops;reference_index=reference_index)

# dim = get_subspace_dimension(subspace)
# indexer = CombinationIndexer(reduce(vcat,collect(sites(subspace.lattice))), get_subspace_info(subspace)...)
# difference_dict = collect_all_conf_differences(indexer)


(sparse([1, 5, 8, 17, 36, 2, 3, 5, 6, 11  …  1810, 1815, 1816, 1818, 1819, 1785, 1804, 1813, 1816, 1820], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2  …  1819, 1819, 1819, 1819, 1819, 1820, 1820, 1820, 1820, 1820], [0.0, 2.0, 2.0, 2.0, 2.0, 0.0, -1.0, 1.0, 1.0, 2.0  …  2.0, -1.0, -1.0, 1.0, 0.0, 2.0, 2.0, 2.0, 2.0, 0.0], 1820, 1820), CombinationIndexer{Coordinate{2, Int64}}(Coordinate{2, Int64}[Coordinate(1, 1), Coordinate(2, 1), Coordinate(3, 1), Coordinate(4, 1), Coordinate(1, 2), Coordinate(2, 2), Coordinate(3, 2), Coordinate(4, 2)], Dict{Tuple{Set{Coordinate{2, Int64}}, Set{Coordinate{2, Int64}}}, Int64}((Set([Coordinate(3, 1), Coordinate(2, 2)]), Set([Coordinate(2, 2), Coordinate(1, 1)])) => 943, (Set([Coordinate(1, 2), Coordinate(2, 1)]), Set([Coordinate(2, 2), Coordinate(1, 1)])) => 775, (Set([Coordinate(1, 1)]), Set([Coordinate(1, 1), Coordinate(3, 2), Coordinate(4, 1)])) => 84, (Set([Coordinate(2, 2)]), Set([Coordinate(3, 1), Coordinate(2, 2), Coordinate(4, 2)])) => 395, (Set([Coordinate(4,

In [9]:
op = create_operator(subspace,:T, kind=1)
r, c, v = findnz(op)
maximum(diff(c))

1

In [73]:
function find_representatives(dim::Int, eig_indices::Vector{Int}, n_eigs::Vector{Int}, 
        mapping::Vector, mapping_sign::Vector)
    # 1<= eig_idx <= n_eigs
    checked_indices = Array{Any}(undef, dim)
    representative_indices = []
    associated_representative = zeros(Int, dim)
    magnitude = []
    periods = [[] for _ in eachindex(eig_indices)]
    for i = 1:dim
        if isassigned(checked_indices, i)
            continue
        end
        # println(i)

        period = ones(Int, length(eig_indices))
        overall_signs = ones(Int, length(eig_indices))

        # finding periods
        for (l, (map_l, sign_l)) in enumerate(zip(mapping, mapping_sign))
            j = map_l[i]
            overall_signs[l] *= sign_l[j]
            while j != i
                period[l] += 1
                j = map_l[j]
                overall_signs[l] *= sign_l[j]
            end
        end

        # searching through remaining states corresponding to representative
        index_matrix = zeros(Int, period...)
        index_matrix[ones(Int, period...)...] = i
        checked_indices[i] = (Tuple(ones(Int, length(period))), 1)
        # println(checked_indices[1:20])
        for indices in Iterators.product([1:k for k in period]...)
            if index_matrix[indices...] != 0
                continue
            end

            prev_indices = collect(indices)
            op_k = 0
            for k in eachindex(indices)
                if indices[k] > 1
                    prev_indices[k] -= 1
                    op_k = k
                    break
                end
            end

            prev_index = index_matrix[prev_indices...]
            j = mapping[op_k][prev_index]
            applied_sign = Int(mapping_sign[op_k][prev_index])
            index_matrix[indices...] = j
            if !isassigned(checked_indices,j)
                checked_indices[j] = (indices, checked_indices[prev_index][2]*applied_sign)
                associated_representative[j] = i
            end
        end

        # computing representative weight
        # mR/N + 1/2 = Z if overall_sign == -1
            # mR + N/2 = Z*N
        # mR/N = Z if overall_sign == 1
        mag = 1
        for (l, (s, p, n_eig, eig_i)) in enumerate(zip(overall_signs,period, n_eigs, eig_indices))
            if p == n_eig 
                continue
            elseif s == -1 && n_eig % (2*p) != 0
                mag *= sum(exp(1im*l*2π*((eig_i-1)*p/n_eig + 0.5)) for l=0:(n_eig ÷ p-1))
            elseif ((eig_i - 1)* p + (s == -1)*n_eig/2) % n_eig ≈ 0
                mag *= n_eig/p
            else
                mag *= 0
                break
            end
        end
        # println(mag)
        if !(mag ≈ 0)
            push!(representative_indices, i)
            associated_representative[i] = -length(representative_indices)
            for (i,p) in enumerate(period)
                push!(periods[i], p)
            end
            push!(magnitude, mag)
        end

        # if i > 10
        #     break
        # end

    end
     

    return checked_indices, representative_indices, associated_representative, magnitude, periods
end

function construct_hamiltonian(
        r, c, v; 
        checked_indices, representative_indices, associated_representative, 
        magnitude, n_eigs, eig_indices
    )
    h = spzeros(ComplexF64, length(representative_indices), length(representative_indices))

    i = 1
    for (in_rep_idx, rep_idx) in enumerate(representative_indices)
        while c[i] < rep_idx
            i += 1
            # if i > 3000
            #     error("hey")
            # end
        end
        # println(in_rep_idx)
        while c[i] == rep_idx
            output_idx = r[i]
            h_val = v[i]
            if associated_representative[output_idx] < 0
                out_rep_idx = abs(associated_representative[output_idx])
                phase = 1
                relative_sign = 1
            elseif associated_representative[output_idx] > 0 && associated_representative[associated_representative[output_idx]] < 0
                out_rep_idx = abs(associated_representative[associated_representative[output_idx]])
                (unitary_distance, relative_sign) = checked_indices[output_idx]
                exp_val = 2π*sum((l-1)*(m-1)/N for (l, N, m) in zip(unitary_distance, n_eigs, eig_indices))
                phase = exp(-1im * exp_val)
            else
                i += 1
                continue
            end
            if (out_rep_idx == 9 && in_rep_idx == 8)|| (out_rep_idx == 8 && in_rep_idx == 9)
                println(i)
                println("($out_rep_idx,$in_rep_idx) - ($(r[i]),$(c[i])):  $h_val, $relative_sign, $phase, $(magnitude[out_rep_idx]), $(magnitude[in_rep_idx])")
            end
            h[out_rep_idx, in_rep_idx] += h_val * relative_sign * phase * sqrt(abs(magnitude[out_rep_idx]/magnitude[in_rep_idx]))
            i += 1
        end
    end
    return h
end
n_eigs = [4,2]
eig_indices = [1,1]
mapping = []
s_mapping = [] 
for kind in 1:2
    op = create_operator(subspace,:T, kind=kind)
    r, c, v = findnz(op)
    push!(mapping, r)
    push!(s_mapping, v)
end
checked_indices, representative_indices, associated_representative, magnitude, periods = find_representatives(size(H)[1], eig_indices, n_eigs, mapping, s_mapping)
new_h = construct_hamiltonian(
    findnz(H)...; 
    checked_indices, representative_indices, associated_representative, 
    magnitude, n_eigs, eig_indices
)

99
(9,8) - (15,14):  1.0, 1, 1, 1, 1
106
(8,9) - (14,15):  1.0, 1, 1, 1, 1
110
(8,9) - (51,15):  1.0, 1, 1.0 - 0.0im, 1, 1


238×238 SparseMatrixCSC{ComplexF64, Int64} with 2125 stored entries:
⎡⢿⡷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⢿⣷⣯⡾⣷⠆⣴⣐⠈⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⣫⡿⠵⣧⣌⣷⣅⠹⠷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠹⠟⢦⣽⣟⣽⣻⣏⠣⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢐⢻⣅⡙⡿⢾⣿⣿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠂⠄⠙⠧⠉⠢⠙⠿⠿⣧⣤⢀⡀⠀⣄⣀⠀⠀⠀⢄⠀⠀⠀⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢛⢵⣷⣜⣣⣪⣫⠀⠀⠀⠀⡛⣬⠡⢐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠶⣹⣟⣽⠘⠚⣤⠀⠀⠤⢁⢂⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⡮⣺⣲⠀⣿⣿⣆⠀⠀⠀⠀⠀⠀⠀⠱⣵⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠈⠙⢻⣶⣶⡶⣴⣤⣀⢖⠁⠃⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢄⠀⠀⠀⡄⠀⠀⢸⡿⣫⣾⣮⢛⡿⠛⣄⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡛⣬⠡⢐⠀⠀⠐⣿⣮⢛⢵⣷⣜⣣⣪⠿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⢁⢂⠛⢄⠀⠀⢠⢜⣿⠋⠶⣹⣟⣽⡜⡞⠕⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢕⣦⠥⠀⠀⢽⣮⡞⣲⠭⣿⣿⡷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙⠁⠀⠀⠀⠈⠙⠑⠁⠙⠋⢻⣶⠀⠀⠐⢄⢖⡄⠐⢄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣦⡀⠑⣽⢾⡵⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⢄⠈⠻⣿⣿⣶⡶⣿⣿⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠵⣕⣤⢸⡿⡿⣯⣾⡿⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⢄⢞⡷⣿⣿⣾⡿⣿⣿⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⡷⎦

In [69]:
associated_representative

1820-element Vector{Int64}:
    0
   -1
   -2
   -3
   -4
    3
    4
    5
    2
   -5
    ⋮
 1754
 1755
 1752
 1753
 1755
 1752
 1753
 1754
 1751

In [64]:
new_h[1:10,1:10]

10×10 SparseMatrixCSC{ComplexF64, Int64} with 32 stored entries:
      ⋅      -2.0+0.0im       ⋅      …          ⋅          ⋅          ⋅    
 -2.0+0.0im       ⋅      -2.0+0.0im             ⋅          ⋅          ⋅    
      ⋅      -2.0+0.0im       ⋅             0.0+0.0im      ⋅          ⋅    
  2.0+0.0im       ⋅       2.0+0.0im             ⋅      0.0+0.0im      ⋅    
      ⋅           ⋅           ⋅         1.41421+0.0im      ⋅          ⋅    
  0.0+0.0im       ⋅           ⋅      …          ⋅      2.0+0.0im  2.0+0.0im
      ⋅       0.0+0.0im       ⋅             0.0+0.0im      ⋅          ⋅    
      ⋅           ⋅       0.0+0.0im             ⋅      2.0+0.0im  2.0+0.0im
      ⋅           ⋅           ⋅             1.0+0.0im      ⋅          ⋅    
      ⋅           ⋅           ⋅             2.0+0.0im      ⋅          ⋅    

In [52]:
s_mapping[1][1]

-1.0

In [50]:
checked_indices[1:20]

20-element Vector{Any}:
 ((1, 1), 1)
 ((1, 1), 1)
 ((4, 1), 1)
 ((2, 1), -1)
 ((2, 1), -1)
 ((3, 1), 1)
 ((1, 1), 1)
 ((1, 1), 1)
 ((1, 1), 1)
 ((2, 2), -1)
 ((1, 2), 1)
 ((4, 1), 1)
 ((4, 1), 1)
 ((4, 1), 1)
 ((1, 2), 1)
 ((3, 1), -1)
 ((4, 2), 1)
 ((3, 1), 1)
 ((3, 1), 1)
 ((1, 2), 1)

In [47]:
eigsolve(H, rand(ComplexF64, size(H)[1]), 2, :SR)

([-15.77460484178573, -13.799445518334002, -13.799445398376875, -13.750267595812932, -13.749317237360199], Vector{ComplexF64}[[3.258017373480503e-18 + 4.374830873108993e-18im, 5.607630971238372e-18 - 1.2177641673713645e-18im, -1.505787961608353e-18 - 3.733708317978046e-18im, 4.001301787632564e-18 - 7.26782532718837e-19im, -6.643026929813875e-19 - 7.283446142989883e-18im, -2.9245354702804744e-18 - 1.106095301620725e-18im, 9.108056673476045e-19 - 3.411966515471725e-18im, -2.3607034453015187e-18 - 1.1970825306354345e-18im, -1.0771488584031165e-17 - 5.4113341508963825e-18im, -3.221986005616563e-18 + 1.1642406998491058e-18im  …  3.3055943990416234e-19 - 1.2059010411892822e-18im, -1.6944444360241768e-18 + 9.30548608835534e-19im, -2.8611084166694385e-18 + 7.183113827778178e-19im, 1.0129384928022539e-18 - 3.807883399995366e-18im, 4.368431866223476e-18 - 2.143523034206705e-18im, -1.0786732256949154e-19 + 3.0937249989111987e-18im, -2.076025673558174e-18 + 9.062668970375568e-19im, -1.521095577716

In [66]:
indexer.inv_comb_dict[7]

(Set{Coordinate{2, Int64}}(), Set(Coordinate{2, Int64}[Coordinate(2, 2), Coordinate(1, 1), Coordinate(4, 1), Coordinate(2, 1)]))

In [15]:
new_h[1:5,1:5]

5×5 SparseMatrixCSC{ComplexF64, Int64} with 8 stored entries:
     ⋅       4.0+0.0im      ⋅               ⋅               ⋅    
 4.0+0.0im       ⋅      0.0+0.0im           ⋅               ⋅    
     ⋅      -2.0+0.0im      ⋅               ⋅               ⋅    
     ⋅           ⋅          ⋅      0.203092+0.0im       0.0+0.0im
     ⋅           ⋅          ⋅           0.0+0.0im  0.203092+0.0im

In [14]:
x = spzeros(size(H)[1])
x[1] = 1
op * x

8008-element SparseVector{Float64, Int64} with 1 stored entry:
  [4]  =  -1.0

In [10]:
maximum(periods)

2

In [ ]:
find_symmetric_basis(ops, [1,1,1])
new_h = operator_subspace(op, bases)

# Krylov benchmarking

In [ ]:
dic = load("/home/jek354/research/ML-signproblem/experimenting/ed/data/Krylov_ED_benchmark.jld2")
plot()


Dict{String, Any} with 3 entries:
  "krylov_times"      => Any[0.000662526, 0.000660164, 0.000946359, 0.000945191…
  "system_size"       => Any[66, 66, 66, 66, 66, 220, 220, 220, 220, 220  …  24…
  "eigen_value_count" => Any[1, 2, 3, 4, 5, 1, 2, 3, 4, 5  …  1, 2, 3, 4, 5, 1,…